# ニューラルネットワーク（Neural Network）
## 数値勾配（Numerical Gradient）を用いた実装

In [1]:
import sys
from pathlib import Path

sys.path.append(str(Path('__file__').resolve().parent.parent.parent.parent))

import numpy as np

from src.modules.scripts.activation_function import sigmoid
from src.modules.scripts.activation_function import softmax
from src.modules.scripts.loss_function import cross_entropy_error
from src.modules.scripts.numeric import numerical_gradient

### データの取得・確認

In [2]:
from keras.datasets import mnist
import pandas as pd

(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.reshape(60000, 784).astype('float32') / 255
X_test = X_test.reshape(10000, 784).astype('float32') / 255
y_train = np.array(pd.get_dummies(y_train))
y_test = np.array(pd.get_dummies(y_test))

print('X_train samples {}'.format(X_train.shape))
print('y_train samples {}'.format(y_train.shape))
print('X_test samples {}'.format(X_test.shape))
print('y_test samples {}'.format(y_test.shape))


Using TensorFlow backend.
/Users/esaki01/.local/share/virtualenvs/ml-notebooks-rAvDjhvL/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/esaki01/.local/share/virtualenvs/ml-notebooks-rAvDjhvL/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/esaki01/.local/share/virtualenvs/ml-notebooks-rAvDjhvL/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(

X_train samples (60000, 784)
y_train samples (60000, 10)
X_test samples (10000, 784)
y_test samples (10000, 10)


### モデルの構築・訓練

In [3]:
class TwoLayerNet(object):
    """2層ニューラルネットワーク（隠れ層が1つ）."""
    
    def __init__(self, input_size, hidden_size, output_size, weight_init=0.01):
        """コンストラクタ.
        
        input_size: 入力層のニューロン数
        hidden_size: 隠れ層のニューロン数
        output_size: 出力層のニューロン数
        weight_init: 重み初期化時のガウス分布のスケール
        """
        # 重みの初期化
        self.params = {}
        self.params['W1'] = weight_init * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)

        self._train_loss_list = []
        self._train_acc_list = []

    def fit(self, x, t, iters_num, batch_size, learning_rate):
        """学習する.
        
        x: 入力データ
        t: ターゲット
        iters_num: 訓練回数
        batch_size: バッチサイズ
        learning_rate: 学習率
        """
        # 1エポックあたりの繰り返し数
        iter_per_epoch = max(x.shape[0] / batch_size, 1)

        for i in range(1, iters_num + 1):
            # ミニバッチの取得
            batch_mask = np.random.choice(x.shape[0], batch_size)
            x_batch = x[batch_mask]
            t_batch = t[batch_mask]

            # 勾配の計算
            grad = self.gradient(x_batch, t_batch)

            # パラメータの更新
            for key in ('W1', 'b1', 'W2', 'b2'):
                self.params[key] -= learning_rate * grad[key]

            # 学習過程の記録
            loss = self.loss(x_batch, t_batch)
            self._train_loss_list.append(loss)
            print('\r{}回目のError: {}'.format(i, loss), end='')

            # 1エポックごとに認識精度を計算
            if i % iter_per_epoch == 0:
                train_acc = self.accuracy(x, t)
                self._train_acc_list.append(train_acc)
                print()
                print('{} epoch, train accuracy: {}'.format(int(i / iter_per_epoch), train_acc))

    def predict(self, x):
        """予測値を出力する.
        
        x: 入力データ
        """
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']

        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2

        return softmax(a2)

    def loss(self, x, t):
        """エラー値を出力する.
        
        x: 入力データ
        t: 教師データ
        """
        y = self.predict(x)
        return cross_entropy_error(y, t)

    def accuracy(self, x, t):
        """精度を求める.
        
        x: 入力データ
        t: 教師データ
        """
        y = np.argmax(self.predict(x), axis=1)
        t = np.argmax(t, axis=1)

        return np.sum(y == t) / float(x.shape[0])

    def gradient(self, x, t):
        """数値微分をする.
        
        x: 入力データ
        t: 教師データ
        """
        loss = lambda W: self.loss(x, t)

        grads = {}
        grads['W1'] = numerical_gradient(loss, self.params['W1'])
        grads['b1'] = numerical_gradient(loss, self.params['b1'])
        grads['W2'] = numerical_gradient(loss, self.params['W2'])
        grads['b2'] = numerical_gradient(loss, self.params['b2'])

        return grads


In [4]:
network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)
network.fit(X_train, y_train, 3000, 100, 0.1)


37回目のError: 2.2955041374850214

KeyboardInterrupt: 